# Climate Data from Home System

In [172]:
from erddapy import ERDDAP
import pandas as pd
import datetime

# for secondary/derived parameters
from metpy.units import units
import metpy.calc as mpcalc

In [173]:
server_url = 'http://raspberrypi.local:8080/erddap'

In [189]:
e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for='MoonFlower'))
print(df['Dataset ID'].values)

['tempest_moonflower_wx' 'channel_1027974_thingspeak'
 'channel_1037066_thingspeak' 'channel_1047747_thingspeak'
 'channel_843357_thingspeak' 'channel_rpi' 'allDatasets']


In [175]:
dataset_id='tempest_moonflower_wx'

try:
    d = ERDDAP(server=server_url,
        protocol='tabledap',
        response='csv'
    )
    d.dataset_id=dataset_id
except HTTPError:
    print('Failed to generate url {}'.format(dataset_id))

try:
    df_m = d.to_pandas(
                index_col='time (UTC)',
                parse_dates=True,
                skiprows=(1,)  # units information can be dropped.
                )
    df_m.sort_index(inplace=True)
    df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

except:
    print(f"something failed in data download {dataset_id}")
    pass

df_m.drop(columns=['device_id', 'bucket_step_minutes', 'wind_lull','wind_interval'],inplace=True)
#stats are all utc driven - but we really want local daily values
df_m=df_m.tz_convert('US/Pacific')

# calculations of various parameters... metpy?
# HDD/CDD, dewpointTemp

In [176]:
df_m['dewpointTemp']=mpcalc.dewpoint_from_relative_humidity(df_m.temperature.values * units.degC,
                                                            df_m.humidity.values * units.percent)
#wetbulb from metpy had issues

In [177]:
df_m['SLP']=df_m.statioinpressure.values * (1+((1013.25/df_m.statioinpressure.values)**((287.05*0.0065)/9.80665)) * (0.0065*87.3)/288.15)**(9.80665/(287.05*0.0065))

In [178]:
df_daily_max = df_m.resample('D').max()
df_daily_min = df_m.resample('D').min()
df_daily_ave = df_m.resample('D').mean()
df_daily_total = df_m.resample('1T').mean().resample('D').sum()
df_m.sample()

,type,wind_avg,wind_gust,wind_dir,statioinpressure,temperature,humidity,lux,uv,solar_radiation,...,strike_distance,strike_count,battery,report_interval,local_daily_precip,precip_final,local_daily_precip_final,precip_analysis_type,dewpointTemp,SLP
time (UTC),,,,,,,,,,,,,,,,,,,,,
2020-07-16 05:30:00-07:00,obs_st,0.0,0.0,0.0,1005.0,13.5,97.0,248,0.01,2.0,...,0.0,0.0,2.62,1.0,0.0,0.0,0.0,2.0,13.033276,1015.462204


In [179]:
use_current_month = True

if use_current_month:
    current_month = datetime.datetime.now().month
else:
    current_month = 6

current_month_grid_data=pd.DataFrame()
current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].temperature
current_month_grid_data = pd.concat([current_month_grid_data,
                                     df_daily_min[df_daily_min.index.month==current_month].temperature.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].temperature.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].dewpointTemp.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].SLP.round(1),
                                     df_daily_total[df_daily_total.index.month==current_month].solar_radiation.round(0),
                                     df_daily_max[df_daily_max.index.month==current_month].uv.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].wind_avg.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].wind_dir.astype(int),
                                     df_daily_max[df_daily_max.index.month==current_month].wind_gust.round(1)
                                    ],axis=1)
current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature','mean_dewpoint','mean SLP','total_solar_radiation','max_uv_index','average speed','average dir','max gust')

In [183]:
current_month_grid_data['station_id'] = 'tempest'

In [184]:
#this should go to erddap
current_month_grid_data.to_csv(f'MoonflowerTempest_2020{str(current_month).zfill(2)}.csv')

In [188]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['color: red' if v else '' for v in is_max]

def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['color: blue' if v else '' for v in is_min]

current_month_grid_data.drop('station_id',axis=1).style.apply(highlight_max).apply(highlight_min).format("{:.2f}")


,max_temperature,min_temperature,mean_temperature,mean_dewpoint,mean SLP,total_solar_radiation,max_uv_index,average speed,average dir,max gust
time (UTC),,,,,,,,,,
2020-07-01 00:00:00-07:00,18.80,10.90,13.00,11.90,1015.60,208604.00,6.60,0.60,149.00,5.00
2020-07-02 00:00:00-07:00,15.20,9.20,12.20,12.00,1017.30,102809.00,2.30,0.50,124.00,4.80
2020-07-03 00:00:00-07:00,18.00,9.10,13.20,11.70,1020.60,192029.00,4.30,0.30,88.00,4.50
2020-07-04 00:00:00-07:00,22.70,10.30,14.60,11.60,1024.30,307947.00,8.70,0.40,74.00,4.30
2020-07-05 00:00:00-07:00,24.40,8.30,15.00,10.60,1023.00,508393.00,6.00,0.50,72.00,6.50
2020-07-06 00:00:00-07:00,22.30,8.50,14.70,10.90,1019.50,435852.00,7.10,0.50,93.00,5.40
2020-07-07 00:00:00-07:00,16.80,11.10,13.40,11.50,1020.40,159360.00,3.50,0.30,94.00,3.80
2020-07-08 00:00:00-07:00,22.70,11.00,15.60,12.50,1020.70,379207.00,6.80,0.40,77.00,4.70
2020-07-09 00:00:00-07:00,17.80,11.90,14.10,12.60,1018.60,169914.00,3.30,0.30,70.00,4.20


In [182]:
### need to manage daily records, monthly records, alltime records

## repeat for each sensor on property

choose month or use current (starting with data subsetting)

In [239]:
use_current_month = False

if use_current_month:
    constraints = {
        'time>=': datetime.datetime.now().strftime('%Y-%m-01T00:00:00Z'),
    }
else:
    constraints = {
        'time>=': '2020-01-01T00:00:00Z',
        'time<=': '2027-02-10T00:00:00Z',
    }

In [240]:
alldatasets=['channel_1027974_thingspeak',
            'channel_1037066_thingspeak',
            'channel_1047747_thingspeak',
            'channel_843357_thingspeak',
            'channel_rpi']
df_all = {}

for dataset_id in alldatasets:
    try:
        d = ERDDAP(server=server_url,
            protocol='tabledap',
            response='csv'
        )
        d.dataset_id=dataset_id
        d.constraints=constraints
        
    except HTTPError:
        print('Failed to generate url {}'.format(dataset_id))

    try:
        df_m = d.to_pandas(
                    index_col='time (UTC)',
                    parse_dates=True,
                    skiprows=(1,)  # units information can be dropped.
                    )
        df_m.sort_index(inplace=True)
        df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

    except:
        print(f"something failed in data download {dataset_id}")
        pass

    #stats are all utc driven - but we really want local daily values
    df_m=df_m.tz_convert('US/Pacific')
    df_all.update({dataset_id:df_m})
    # calculations of various parameters... metpy?
    # HDD/CDD, dewpointTemp

In [241]:
for k,v in enumerate(df_all):
    print(df_all[v].keys())

Index(['entry_id', 'temperature'], dtype='object')
Index(['entry_id', 'temperature', 'RH_Percent'], dtype='object')
Index(['entry_id', 'temperature_internal', 'temperature_external',
       'RH_Percent'],
      dtype='object')
Index(['entry_id', 'temperature', 'RH_Percent', 'Barotemperature', 'SLP'], dtype='object')
Index(['humidity', 'Average_SLP', 'temperature', 'eCO2', 'eVOC'], dtype='object')


In [247]:
use_current_month = False

if use_current_month:
    current_month = datetime.datetime.now().month
else:
    current_month = 1
    
for k,v in enumerate(df_all):
    print(df_all[v].keys())
    df_daily_max = df_all[v].resample('D').max()
    df_daily_min = df_all[v].resample('D').min()
    df_daily_ave = df_all[v].resample('D').mean()
    if ('RH_Percent' in df_all[v].keys()) and ('temperature' in df_all[v].keys()) and (not 'Barotemperature' in df_all[v].keys()):
        print(f"processing {v} :0")
        current_month_grid_data=pd.DataFrame()
        current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].temperature
        current_month_grid_data = pd.concat([current_month_grid_data,
                                             df_daily_min[df_daily_min.index.month==current_month].temperature.round(1),
                                             df_daily_ave[df_daily_ave.index.month==current_month].temperature.round(1),
                                             df_daily_ave[df_daily_ave.index.month==current_month].RH_Percent.round(1),
                                            ],axis=1)
        current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature','mean_humidity')
        current_month_grid_data['station_id'] = v    
        current_month_grid_data.to_csv(f'{v}_2020{str(current_month).zfill(2)}.csv')
    elif (not 'RH_Percent' in df_all[v].keys()) and ('temperature' in df_all[v].keys()) and (not 'Barotemperature' in df_all[v].keys()):
        print(f"processing {v} :1")
        current_month_grid_data=pd.DataFrame()
        current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].temperature
        current_month_grid_data = pd.concat([current_month_grid_data,
                                             df_daily_min[df_daily_min.index.month==current_month].temperature.round(1),
                                             df_daily_ave[df_daily_ave.index.month==current_month].temperature.round(1),
                                            ],axis=1)
        current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature')
        current_month_grid_data['station_id'] = v
        
        #this should go to erddap
        current_month_grid_data.to_csv(f'{v}_2020{str(current_month).zfill(2)}.csv')
    elif ('RH_Percent' in df_all[v].keys()) and ('temperature' in df_all[v].keys()) and ('Barotemperature' in df_all[v].keys()):
        print(f"processing {v} :2")
        current_month_grid_data=pd.DataFrame()
        current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].Barotemperature
        current_month_grid_data = pd.concat([current_month_grid_data,
                                             df_daily_min[df_daily_min.index.month==current_month].Barotemperature.round(1),
                                             df_daily_ave[df_daily_ave.index.month==current_month].Barotemperature.round(1),
                                            ],axis=1)
        current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature')
        current_month_grid_data['station_id'] = v
        
        #this should go to erddap
        current_month_grid_data.to_csv(f'{v}_2020{str(current_month).zfill(2)}.csv')
    else:
        print(f"passing {v} :3")
        pass

Index(['entry_id', 'temperature'], dtype='object')
processing channel_1027974_thingspeak :1
Index(['entry_id', 'temperature', 'RH_Percent'], dtype='object')
processing channel_1037066_thingspeak :0
Index(['entry_id', 'temperature_internal', 'temperature_external',
       'RH_Percent'],
      dtype='object')
passing channel_1047747_thingspeak :3
Index(['entry_id', 'temperature', 'RH_Percent', 'Barotemperature', 'SLP'], dtype='object')
processing channel_843357_thingspeak :2
Index(['humidity', 'Average_SLP', 'temperature', 'eCO2', 'eVOC'], dtype='object')
processing channel_rpi :1
